# Top 14 Rugby ?

par Benjamin Cerf, Tristan Delhaye & Mathieu Roig

<a id="intro"></a>
# Introduction

à rédiger

<a id="sommaire"></a>
### Sommaire
- [Introduction](#intro)
  - [Sommaire](#sommaire)
  - [Installation](#installation)
- [Données](#données)
  - [Web-Scraping](#scraping)
  - [Nettoyage](#nettoyage)
  - [Regroupement](#regroupement)
  - [Mapping des noms de clubs](#mapping)
- [Visualisation](#visual)
  - [Statistiques](#stats)
  - [Analyse](#analyse)
- [Modèle prédictif](#predict)
- [Conclusion](#conclu)

<a id="installation"></a>
### Installation

In [5]:
#packages:
from io import StringIO
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import sys
import subprocess
import os
import numpy as np
import matplotlib.pyplot as plt

#lxml
subprocess.check_call([sys.executable, "-m", "pip", "install", "lxml"])

# ajouter pour partie predict

0

<a id="données"></a>
# Données

<a id="scraping"></a>
### Web-Scraping

Nous récupérons sur Wikipédia les données sur le championnat de France de rugby à XV de 2016 à aujourd'hui. Chaque saison a sa propre page wikipédia dédiée et la présentation des résultats peut légèrement différer en fonction de l'année: voir ici la saison [2016-2017](https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2016-2017) par rapport à la saison [2024-2025](https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2024-2025).

In [6]:
urls = [f"https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_{year}-{year+1}"
    for year in range(2016, 2025)]
print(urls)

['https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2016-2017', 'https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2017-2018', 'https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2018-2019', 'https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2019-2020', 'https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2020-2021', 'https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2021-2022', 'https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2022-2023', 'https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2023-2024', 'https://fr.wikipedia.org/wiki/Championnat_de_France_de_rugby_%C3%A0_XV_2024-2025']


Nous utilisons donc plusieurs versions d'une même fonction data dans scrapData.py qui récupère les tableaux de présentation générale, du classement, de son évolution, les résultats détaillés par match, et aussi la forme des équipes en fonction des jours de match.

In [7]:
sys.path.append(os.path.abspath('./scripts'))
from scrapData import data2325, data2223, data2122, data2021, data1920, data1619

tab1617 = data1619(urls[0])
tab1718 = data1619(urls[1])
tab1819 = data1619(urls[2])
tab1920 = data1920(urls[3])
tab2021 = data2021(urls[4])
tab2122 = data2122(urls[5])
tab2223 = data2223(urls[6])
tab2324 = data2325(urls[7])
tab2425 = data2325(urls[8])

<a id="nettoyage"></a>
### Nettoyage

La fonction nettoyage de cleanData.py uniformise les données du scraping. Les noms des équipes sont nettoyés, on se débarrasse des notes, et les classements sont tous mis sous forme numérique (ex: 1er -> 1, 2ème -> 2...). Les valeurs manquantes sont aussi remplacées par un tiret.

In [8]:
from cleanData import nettoyage

nettoyage(*tab1617)
nettoyage(*tab1718)
nettoyage(*tab1819)
nettoyage(*tab1920)
nettoyage(*tab2021)
nettoyage(*tab2122)
nettoyage(*tab2223)
nettoyage(*tab2324)
nettoyage(*tab2425)

<a id="regroupement"></a>
### Regroupement

Chaque tab est en réalité une liste de 5 dataframes. Nous allons donc pour chaque tableau fusionner les résultats des 9 dernières années, sauf celui des résultats globaux. 

In [9]:
# On va maintenant fusionner les tableaux, on change le nom pour faciliter la boucle :
tab1 = tab1617
tab2 = tab1718
tab3 = tab1819
tab4 = tab1920
tab5 = tab2021
tab6 = tab2122
tab7 = tab2223
tab8 = tab2324
tab9 = tab2425

# On crée une fonction qui permet d'ajouter une colonne année à chaque tableau pour bien pouvoir les séparer par année si besoin après
def ajout_an(df,i):
   df["année"] = 2016 + i
   return df

# On va faire maintenant une boucle pour concaténer les tableaux entre eux :
tableauglobal = [pd.DataFrame() for _ in range(5)]
# On remarquera que l'on ne prend pas le tableau résultat global car les équipes du championnat changent chaque année donc ce n'est pas possible de le concaténer

for j in range(1, 10):
    for i in range(5):
        if i == 2:
            pass  # on ne récupère pas le tableau résultat
        else:
         tableauglobal[i] = pd.concat([tableauglobal[i], ajout_an(eval(f"tab{j}")[i], j - 1)], ignore_index=True)

tab_presentation_global = tableauglobal[0]
tab_classement_global = tableauglobal[1]
tab_evolution_classement_global = tableauglobal[3]
tab_forme_global = tableauglobal[4]
tab_presentation_global['Budget en M€'] = pd.to_numeric(tab_presentation_global['Budget en M€'], errors='coerce')
tab_presentation_global['Classement précédent'] = pd.to_numeric(tab_presentation_global['Classement précédent'], errors='coerce')


<a id="mapping"></a>
### Mapping des noms de clubs

Enfin certaines équipes n'ont pas toujours leur nom écrit de la même façon.

In [12]:
presentation = tab_presentation_global
classement = tab_classement_global
evolution = tab_evolution_classement_global
forme = tab_forme_global

# Fusion des différentes données en un seul data frame
# On va fusionner les tableaux en se servant du club et de l'année comme clef
# On commence déjà par uniformiser le nom de la colonne 'Club':
evolution.rename(columns={'Equipes/Journées': 'Club'}, inplace=True)
forme.rename(columns={'Equipes/Journées': 'Club'}, inplace=True)

# En fait en faisant la fusion on se rend compte qu'un même club est appelé de manière différentes selon le tableau
# On va donc harmoniser ça
# On trouve les différences d'appelations grâce au code suivant : 
# Clés uniques dans chaque DataFrame
# keys_presentation = set(presentation[['Club', 'année']].itertuples(index=False, name=None))
# keys_classement = set(classement[['Club', 'année']].itertuples(index=False, name=None))
# keys_evolution = set(evolution[['Club', 'année']].itertuples(index=False, name=None))
# keys_forme = set(forme[['Club', 'année']].itertuples(index=False, name=None))
# Lignes présentes uniquement dans presentation
# diff_presentation = keys_presentation - keys_classement
# print("Lignes dans presentation mais pas dans classement :", diff_presentation)
#  # Lignes présentes uniquement dans classement
# diff_classement = keys_classement - keys_presentation
#  print("Lignes dans classement mais pas dans presentation :", diff_classement)
# Le premier tableau utilise la dénomination 'Stade français' tandis que le second 'Stade français Paris'
# On procède de même avec les différents tableaux pour observer toutes les appelations différentes (on le fait par rapport à classement)
# On répertorie alors ces appelations différentes pour pouvoir par la suite uniformiser tout les noms : 

mapping = {
    'Paris' : 'Stade français Paris',
    'Stade français': 'Stade français Paris',
    'Clermont': 'ASM Clermont',
    'La Rochelle': 'Stade rochelais',
    'Toulouse': 'Stade toulousain',
    'Bayonne': 'Aviron bayonnais',
    'Brive': 'CA Brive',
    'Montpellier': 'Montpellier HR',
    'Toulon': 'RC Toulon',
    'Castres': 'Castres olympique',
    'Pau': 'Section paloise',
    'Agen': 'SU Agen',
    'Grenoble': 'FC Grenoble',
    'Oyonnax': 'US Oyonnax',
    'Perpignan': 'USA Perpignan',
    'Bordeaux-Bègles': 'Union Bordeaux Bègles',
    'Bordeaux Bègles' : 'Union Bordeaux Bègles',
    'Lyon' : 'Lyon OU'
}

# On a plus qu'à uniformiser les noms dans tout les tableaux
presentation['Club'] = presentation['Club'].replace(mapping)
classement['Club'] = classement['Club'].replace(mapping)
forme['Club'] = forme['Club'].replace(mapping)
evolution['Club'] = evolution['Club'].replace(mapping)

print(presentation.describe())
print(classement.describe())
print(forme.describe())
print(evolution.describe())

       Dernière montée  Budget en M€  Classement précédent      Capacité  \
count       126.000000    112.000000            126.000000    121.000000   
mean       1996.460317     27.215268              7.714286  19528.446281   
std          34.203544      7.655976              4.410636   7102.591448   
min        1907.000000     12.320000              1.000000  11400.000000   
25%        1997.000000     22.750000              4.000000  14593.000000   
50%        2010.000000     27.190000              7.500000  18069.000000   
75%        2016.000000     32.000000             11.000000  20000.000000   
max        2024.000000     49.000000             17.000000  35052.000000   

             année  
count   126.000000  
mean   2020.000000  
std       2.592296  
min    2016.000000  
25%    2018.000000  
50%    2020.000000  
75%    2022.000000  
max    2024.000000  
             Rang           J           V           N           D          Em  \
count  126.000000  126.000000  126.000000  12

In [ ]:
# fixer les données nettoyées dans SSPcloud ?
ou alors juste exporter 
#import s3fs
#MY_BUCKET =

<a id="visual"></a>
# Visualisation

<a id="stats"></a>
### Statistiques


<a id="analyse"></a>
### Analyse

<a id="predict"></a>
# Modèle prédictif

<a id="conclu"></a>
# Conclusion

à rédiger, rester réaliste et dire ce qu'on aurait pu faire de mieux